In [8]:
import pytorch_lightning as pl 

class FourLayerClassifier(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.hparams = hparams
        self.save_hyperparameters()        
        self.fc1 = nn.Linear(self.hparams["input_size"], 420)
        self.fc2 = nn.Linear(420, 420)
        self.fc3 = nn.Linear(420, 420)
        self.fc4 = nn.Linear(420, self.hparams["targets"])
        self.dropout = nn.Dropout(0.4)

        self.train_acc = pl.metrics.Accuracy()
        self.valid_acc = pl.metrics.Accuracy()
        self.test_acc = pl.metrics.Accuracy()

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('train_loss', loss)
        self.train_acc(y_hat, y)
        self.log('train_acc', self.train_acc, on_step=True, on_epoch=False)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('valid_loss', loss)
        self.valid_acc(y_hat, y)
        self.log('valid_acc', self.valid_acc, on_step=True, on_epoch=True)


    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('test_loss', loss)
        self.test_acc(y_hat, y)
        self.log('test_acc', self.test_acc, on_step=True, on_epoch=True)


    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), self.hparams["learning_rate"])


In [9]:
import torch
from torch import nn
from torch.nn import functional as F
from torchvision import datasets, models, transforms
from torchvision.datasets import MNIST

from privacyraven.utils.data import get_mnist_loaders
from privacyraven.utils.model_creation import (convert_to_inference,
                                               set_hparams, train_and_test)

def train_four_layer_mnist_model(
    transform=None,
    batch_size=100,
    num_workers=4,
    rand_split_val=None,
    gpus=1,
    max_epochs=8,
    learning_rate=1e-3,
):
    input_size = 784
    targets = 10
    if transform is None:
        transform = transforms.Compose(
            [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
        )
    if rand_split_val is None:
        rand_split_val = [55000, 5000]

    # Establish hyperparameters and DataLoaders
    hparams = set_hparams(
        transform,
        batch_size,
        num_workers,
        gpus,
        max_epochs,
        learning_rate,
        input_size,
        targets,
    )

    train_dataloader, val_dataloader, test_dataloader = get_mnist_loaders(hparams)
    
    # Train, test, and convert the model to inference
    mnist_model = train_and_test(
        FourLayerClassifier, train_dataloader, val_dataloader, test_dataloader, hparams
    )
    mnist_model = convert_to_inference(mnist_model)
    return mnist_model

new_model = train_four_layer_mnist_model()

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type     | Params
---------------------------------------
0 | fc1       | Linear   | 329 K 
1 | fc2       | Linear   | 176 K 
2 | fc3       | Linear   | 176 K 
3 | fc4       | Linear   | 4.2 K 
4 | dropout   | Dropout  | 0     
5 | train_acc | Accuracy | 0     
6 | valid_acc | Accuracy | 0     
7 | test_acc  | Accuracy | 0     
---------------------------------------
687 K     Trainable params
0         Non-trainable params
687 K     Total params


Validation sanity check: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validating: |          | 0/? [00:00<?, ?it/s]

Validating: |          | 0/? [00:00<?, ?it/s]

...

Validating: |          | 0/? [00:00<?, ?it/s]

Validating: |          | 0/? [00:00<?, ?it/s]

Validating: |          | 0/? [00:00<?, ?it/s]

Testing: |          | 0/? [00:00<?, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.9794, device='cuda:0'),
 'test_acc_epoch': tensor(0.9794, device='cuda:0'),
 'test_loss': tensor(0.0735, device='cuda:0'),
 'valid_acc': tensor(0.9764, device='cuda:0'),
 'valid_acc_epoch': tensor(0.9764, device='cuda:0'),
 'valid_loss': tensor(0.0780, device='cuda:0')}
--------------------------------------------------------------------------------


In [10]:
import privacyraven as pr

from privacyraven.utils.data import get_emnist_data
from privacyraven.extraction.core import ModelExtractionAttack
from privacyraven.utils.query import get_target
from privacyraven.models.victim import train_four_layer_mnist_victim
from privacyraven.models.four_layer import FourLayerClassifier

# Create a query function for a target PyTorch Lightning model
#model = train_four_layer_mnist_model()


def query_mnist(input_data):
    # PrivacyRaven provides built-in query functions
    return get_target(new_model, input_data, (1, 28, 28, 1))

# Obtain seed (or public) data to be used in extraction
emnist_train, emnist_test = get_emnist_data()

# Run a model extraction attack
attack = ModelExtractionAttack(
    query_mnist,
    100,
    (1, 28, 28, 1),
    10,
    (3, 1, 28, 28),
    "copycat",
    FourLayerClassifier,
    784,
    emnist_train,
    emnist_test
)

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type     | Params
---------------------------------------
0 | fc1       | Linear   | 329 K 
1 | fc2       | Linear   | 176 K 
2 | fc3       | Linear   | 176 K 
3 | fc4       | Linear   | 4.2 K 
4 | dropout   | Dropout  | 0     
5 | train_acc | Accuracy | 0     
6 | valid_acc | Accuracy | 0     
7 | test_acc  | Accuracy | 0     
---------------------------------------
687 K     Trainable params
0         Non-trainable params
687 K     Total params


Synthesis complete
Synthetic Data Generated


Validation sanity check: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validating: |          | 0/? [00:00<?, ?it/s]

...

Validating: |          | 0/? [00:00<?, ?it/s]

Validating: |          | 0/? [00:00<?, ?it/s]

Validating: |          | 0/? [00:00<?, ?it/s]

Testing: |          | 0/? [00:00<?, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': tensor(0.3400, device='cuda:0'),
 'test_acc_epoch': tensor(0.3400, device='cuda:0'),
 'test_loss': tensor(4.6119, device='cuda:0'),
 'valid_acc': tensor(0.1500, device='cuda:0'),
 'valid_acc_epoch': tensor(0.1500, device='cuda:0'),
 'valid_loss': tensor(6.8039, device='cuda:0')}
--------------------------------------------------------------------------------
Out of 100 data points, the models agreed upon 27.
